# Testing step by step

In [ ]:
# Imports
import numpy as np
import os
import time
import sys
import matplotlib.pyplot as plt
from astropy import stats
from shutil import copy
import pandas as pd

from config import CONFIG_INI
from mpfit import mpfit
from limb_darkening import limb_dark_fit
import hstmarg

In [ ]:
# Data setup
localDir = CONFIG_INI.get('data_paths', 'local_path')
outDir = os.path.join(localDir, CONFIG_INI.get('data_paths', 'output_path'))
curr_model = CONFIG_INI.get('data_paths', 'current_model')
dataDir = os.path.join(localDir, os.path.join(localDir, CONFIG_INI.get('data_paths', 'data_path')), curr_model)

# READ in the txt file for the lightcurve data
x, y, err, sh = np.loadtxt(os.path.join(dataDir, 'W17_white_lightcurve_test_data.txt'), skiprows=7, unpack=True)
wavelength = np.loadtxt(os.path.join(dataDir, 'W17_wavelength_test_data.txt'), skiprows=3)

# Limb darkening parameters - user input
ld_model = CONFIG_INI.get('limb_darkening', 'ld_model')
FeH = CONFIG_INI.getfloat('limb_darkening', 'metallicity')
Teff = CONFIG_INI.getfloat('limb_darkening', 'Teff')
logg = CONFIG_INI.getfloat('limb_darkening', 'logg')

# More user input
grat = CONFIG_INI.get('technical_parameters', 'grating')
grid_selection = CONFIG_INI.get('technical_parameters', 'grid_selection')
run_name = CONFIG_INI.get('technical_parameters', 'run_name')
plotting = CONFIG_INI.get('technical_parameters', 'plotting')

# Planet parameters
rl = CONFIG_INI.getfloat('planet_parameters', 'rl')             # Rp/R* estimate
epoch = CONFIG_INI.getfloat('planet_parameters', 'epoch')       # in MJD
inclin = CONFIG_INI.getfloat('planet_parameters', 'inclin')     # this is converted into radians in the subroutine
ecc = CONFIG_INI.getfloat('planet_parameters', 'ecc')           # set to zero and not used when circular
omega = CONFIG_INI.getfloat('planet_parameters', 'omega')       # set to zero and not used when circular
Per = CONFIG_INI.getfloat('planet_parameters', 'Per')           # in days, converted to seconds in subroutine
aor = CONFIG_INI.getfloat('planet_parameters', 'aor')           # a/R* converted to system density for the subroutine

# Setting constants and preparing inputs for claculations
dtosec = CONFIG_INI.getfloat('constants', 'dtosec')     # conversion from days to seconds
big_G = CONFIG_INI.getfloat('constants', 'big_G')       # gravitational constant

persec = Per * dtosec
constant1 = (big_G * persec * persec / np.float32(4. * np.pi * np.pi)) ** (1. / 3.)
MsMpR = (aor / constant1) ** 3.

# Put data parameters in list
data_params = [rl, epoch, inclin, MsMpR, ecc, omega, Per, FeH, Teff, logg]

In [ ]:
# DEFINE LIMB DARKENING DIRECTORY, WHICH IS INSIDE THIS PACKAGE
limbDir = os.path.join('..', 'Limb-darkening')

# READ THE CONSTANTS
Gr = CONFIG_INI.getfloat('constants', 'big_G')
day_to_sec = CONFIG_INI.getfloat('constants', 'dtosec')
HST_period = CONFIG_INI.getfloat('constants', 'HST_period')

# We want to keep the raw data as is, so we generate helper arrays that will get changed from model to model
img_date = x    # time array
img_flux = y    # flux array
img_err = err   # error array
img_sh = sh     # shift in position array
nexposure = len(img_date)   # Total number of exposures in the observation

# READ IN THE PLANET STARTING PARAMETERS
# data_params = [rl, epoch, inclin, MsMpR, ecc, omega, Per, FeH, Teff, logg]   # Description
rl = data_params[0]                             # Rp/R* estimate
epoch = data_params[1]                          # center of transit time in MJD
inclin = data_params[2] * ((2 * np.pi) / 360)   # inclination, converting it to radians
MsMpR = data_params[3]                          # density of the system
ecc = data_params[4]                            # eccentricity
omega = data_params[5] * ((2 * np.pi) / 360)    # orbital omega, converting it to radians
Per = data_params[6] * day_to_sec               # period in seconds
constant1 = ((Gr * np.square(Per)) / (4 * np.square(np.pi))) ** (1 / 3)
aval = constant1 * (MsMpR) ** (1 / 3)   # NOT-REUSED

flux0 = img_flux[0]   # first flux data point
T0 = img_date[0]      # first time data point

# SET THE STARTING PARAMETERS FOR THE SYSTEMATIC MODELS
m_fac = 0.0  # Linear Slope
HSTP1 = 0.0  # Correct HST orbital phase
HSTP2 = 0.0  # Correct HST orbital phase^2
HSTP3 = 0.0  # Correct HST orbital phase^3
HSTP4 = 0.0  # Correct HST orbital phase^4
xshift1 = 0.0  # X-shift in wavelength
xshift2 = 0.0  # X-shift in wavelength^2
xshift3 = 0.0  # X-shift in wavelength^3
xshift4 = 0.0  # X-shift in wavelength^4

# =======================
# LIMB DARKENING
# NEW: Implement a suggestion for the user to use 3D if his parameters match the options available in the 3D models

M_H = data_params[7]    # metallicity
Teff = data_params[8]   # effective temperature
logg = data_params[9]   # log(g), gravitation

uLD, c1, c2, c3, c4, cp1, cp2, cp3, cp4, aLD, bLD = limb_dark_fit(grat, wavelength, M_H, Teff,
                                                                       logg, limbDir, ld_model)
# =======================

# PLACE ALL THE PRIORS IN AN ARRAY
# p0 =        [0,    1,     2,      3,     4,    5,    6,    7,  8,  9,  10, 11, 12,  13,    14,    15,    16,    17,     18,      19,      20,      21   ]
p0 = np.array([rl, flux0, epoch, inclin, MsMpR, ecc, omega, Per, T0, c1, c2, c3, c4, m_fac, HSTP1, HSTP2, HSTP3, HSTP4, xshift1, xshift2, xshift3, xshift4])

# Create an array with the names of the priors
p0_names = np.array(['rl', 'flux0', 'epoch', 'inclin', 'MsMpR', 'ecc', 'omega', 'Per', 'T0', 'c1', 'c2', 'c3', 'c4',
                     'm_fac', 'HSTP1', 'HSTP2', 'HSTP3', 'HSTP4', 'xshift1', 'xshift2', 'xshift3', 'xshift4'])

# Create a dictionary for easier use in calculations
p0_dict = {key: val for key, val in zip(p0_names, p0)}

# SELECT THE SYSTEMATIC GRID OF MODELS TO USE
# 1 in the grid means the parameter is fixed, 0 means it is free
grid = hstmarg.wfc3_systematic_model_grid_selection(grid_selection)
nsys, nparams = grid.shape   # nsys = number of systematic models, nparams = number of parameters

#  SET UP THE ARRAYS

# save arrays for the first step through to get the err inflation
w_scatter = np.zeros(nsys)
w_params = np.zeros((nsys, nparams))   # p0 parameters, but for all the systems in one single array, so that we can acces each one of the individually during the second fit

#################################
#           FIRST FIT           #
#################################

print('\n 1ST FIT \n')
print(
    'The first run through the data for each of the WFC3 stochastic models outlined in Table 2 of Wakeford et '
    'al. (2016) is now being preformed. Using this fit we will scale the uncertainties you input to incorporate '
    'the inherent scatter in the data for each model.')

# Loop over all systems (= parameter combinations)
for s in range(nsys):
    print('\n################################')
    print('SYSTEMATIC MODEL {} of {}'.format(s+1, nsys))
    systematics = grid[s, :]
    print('Systematics - fixed and free parameters:')
    print_dict = {name: fix for name, fix in zip(p0_names, systematics)}
    print(print_dict)
    print(systematics)
    print('  ')

    # Displaying img_date in terms of HST PHASE, on an interval between -0.5 and 0.5
    HSTphase = (img_date - p0_dict['T0']) / HST_period   # make phase (~time) array start at 0 by subtracting first observation time, convert in units of HST phase by dividing through one HST period
    phase2 = np.floor(HSTphase)       # identify where phase is between 0-1, between 1-2, between 2-3 and over 3
    HSTphase = HSTphase - phase2      # make phase be in interval from 0 to 1
    k = np.where(HSTphase > 0.5)[0]   # figure out where phase is bigger than 0.5
    HSTphase[k] -= 1.0                # and where it is bigger than 0.5 indeed, subtract on to get to interval [-0.5, 0.5]

    # Displaying img_date in terms of PLANET PHASE, on interval between -0.5 and 0.5
    phase = (img_date - p0_dict['epoch']) / (p0_dict['Per'] / day_to_sec)   # make center of transit time by subtracting 'epoch' from img_date, convert in units of planet phase by dividing py planet period, convert to seconds
    phase2 = np.floor(phase)          # identify integer intervals of phase (like above)
    phase = phase - phase2            # make phase be in interval from 0 to 1
    a = np.where(phase > 0.5)[0]      # figure out where phase is bigger than 0.5
    phase[a] -= 1.0                   # and where it is bigger than 0.5 indeed, subtract on to get to interval [-0.5, 0.5]

    ###############
    # MPFIT - ONE #
    ###############

    # Create two dictionaries in which each parameter in p0 gets some extra parameters assigned, which we then feed
    # info mpfit. This dictionary has the sole purpose of preparing the input data for mpfit in such a way that
    # it works.
    parinfo = []
    for i, value in enumerate(p0):
        info = {'value': 0., 'fixed': 0, 'limited': [0, 0], 'limits': [0., 0.]}
        info['value'] = value
        info['fixed'] = systematics[i]
        parinfo.append(info)
    fa = {'x': img_date, 'y': img_flux, 'err': err, 'sh': sh}

    print('\nSTART MPFIT\n')
    mpfit_result = mpfit(hstmarg.transit_circle, functkw=fa, parinfo=parinfo, quiet=1)
    print('\nTHIS ROUND OF MPFIT IS DONE\n')

    # Count free parameters by figuring out how many zeros we have in the current systematics
    nfree = sum([not p['fixed'] for p in parinfo])

    # The python mpfit does not populate the covariance matrix correctly so mpfit_result.perror is not correct
    # the mpfit_result.covar is filled sequentially by row with the values of only free parameters, this works if
    # all parameters are free but not if some are kept fixed.  The code below should work to get the proper error
    # values i.e. what should be the diagonals of the covariance.

    test_err1 = mpfit_result.perror  # this is how it should be done if it was right

    pcerror = np.zeros_like(mpfit_result.perror)
    pcerror[:nfree] = np.sqrt(
        np.diag(mpfit_result.covar.flatten()[:nfree ** 2].reshape(nfree, nfree)))  # this might work...

    bestnorm = mpfit_result.fnorm  # chi squared of resulting fit

     # Redefine all of the parameters given the MPFIT output
    w_params[s, :] = mpfit_result.params
    # Populate parameters with fits results
    p0 = w_params[s, :]
    # Recreate the dictionary
    p0_dict = {key: val for key, val in zip(p0_names, p0)}

    # Populate some errors from pcerror array
    # pcerror = [rl_err, flux0_err, epoch_err, inclin_err, msmpr_err, ecc_err, omega_err, per_err, T0_err,
    #           c1_err, c2_err, c3_err, c4_err, m_err, hst1_err, hst2_err, hst3_err, hst4_err, sh1_err, sh2_err,
    #           sh3_err, sh4_err]
    rl_err = pcerror[0]

    # Recalculate a/R* (actually the constant for it) based on the new MsMpR value which may have been fit in the routine.
    constant1 = (Gr * p0_dict['Per'] * p0_dict['Per'] / (4 * np.pi * np.pi)) ** (1 / 3.)

    print('\nTRANSIT DEPTH rl in model {} of {} = {} +/- {}, centered at  {}'.format(s+1, nsys, p0_dict['rl'], rl_err, p0_dict['epoch']))

    # OUTPUTS
    # Re-Calculate each of the arrays dependent on the output parameters
    phase = (img_date - p0_dict['epoch']) / (p0_dict['Per'] / day_to_sec)
    phase2 = np.floor(phase)
    phase = phase - phase2
    a = np.where(phase > 0.5)[0]
    phase[a] = phase[a] - 1.0

    HSTphase = (img_date - p0_dict['T0']) / HST_period
    phase2 = np.floor(HSTphase)
    HSTphase = HSTphase - phase2
    k = np.where(HSTphase > 0.5)[0]
    HSTphase[k] = HSTphase[k] - 1.0

    # ...........................................
    # TRANSIT MODEL fit to the data
    # Calculate the impact parameter based on the eccentricity function
    b0 = hstmarg.impact_param(p0_dict['Per'], p0_dict['MsMpR'], phase, p0_dict['inclin'])

    mulimb01, mulimbf1 = hstmarg.occultnl(p0_dict['rl'], p0_dict['c1'], p0_dict['c2'], p0_dict['c3'], p0_dict['c4'], b0)

    systematic_model = hstmarg.sys_model(phase, HSTphase, sh, p0_dict['m_fac'], p0_dict['HSTP1'], p0_dict['HSTP2'], p0_dict['HSTP3'],
                                         p0_dict['HSTP4'], p0_dict['xshift1'], p0_dict['xshift2'],
                                         p0_dict['xshift3'], p0_dict['xshift4'])

    # Calculate final form of the model fit
    w_model = mulimb01 * p0_dict['flux0'] * systematic_model   # see Wakeford et al. 2016, Eq. 1
    # Calculate the residuals
    w_residuals = (img_flux - w_model) / p0_dict['flux0']
    # Calculate more stuff
    corrected_data = img_flux / (p0_dict['flux0'] * systematic_model)
    w_scatter[s] = np.std(w_residuals)
    print('Scatter on the residuals = {}'.format(w_scatter[s]))   # this result is rather different to IDL result

In [ ]:
plt.plot(phase, mulimb01)
plt.show()

In [ ]:
print(test_err)
print(pcerror)

In [ ]:
################################
#          SECOND FIT          #
################################

print('..........................................')
print('\n 2ND FIT \n')
print('Each systematic model will now be re-fit with the previously determined parameters serving as the new starting points.')

# Initializing arrays for each systematic model, which we will save once we got through all systems with two fits.
sys_stats = np.zeros((nsys, 5))                 # stats       # NEW: why 5? (trying to get rid of hard coded things)
sys_date = np.zeros((nsys, nexposure))          # img_date
sys_phase = np.zeros((nsys, nexposure))         # phase
sys_rawflux = np.zeros((nsys, nexposure))       # raw lightcurve flux
sys_rawflux_err = np.zeros((nsys, nexposure))   # raw lightcurve flux error
sys_flux = np.zeros((nsys, nexposure))          # corrected lightcurve flux
sys_flux_err = np.zeros((nsys, nexposure))      # corrected lightcurve flux error
sys_residuals = np.zeros((nsys, nexposure))     # residuals
sys_model = np.zeros((nsys, 4000))              # smooth model       # NEW: why 4000?
sys_model_phase = np.zeros((nsys, 4000))        # smooth phase       # NEW: why 4000?
sys_systematic_model = np.zeros((nsys, nexposure))  # systematic model
sys_params = np.zeros((nsys, nparams))          # parameters
sys_params_err = np.zeros((nsys, nparams))      # parameter errors
sys_depth = np.zeros(nsys)                      # depth
sys_depth_err = np.zeros(nsys)                  # depth error
sys_epoch = np.zeros(nsys)                      # transit time
sys_epoch_err = np.zeros(nsys)                  # transit time error
sys_evidenceAIC = np.zeros(nsys)                # evidence AIC
sys_evidenceBIC = np.zeros(nsys)                # evidence BIC

for s in range(nsys):
    print('\n################################')
    print('SYSTEMATIC MODEL {} of {}'.format(s+1, nsys))
    systematics = grid[s, :]
    print_dict = {name: fix for name, fix in zip(p0_names, systematics)}
    print(print_dict)
    print(systematics)
    print('  ')

    # Rescale the err array by the standard deviation of the residuals from the fit.
    err *= (1.0 - w_scatter[s])   # w_scatter are residuals
    # Reset the arrays and start again. This is to ensure that we reached a minimum in the chi-squared space.
    p0 = w_params[s, :]   # populate with results from first run FOR THE SYSTEM nsys WE'RE CURRENTLY IN
    # Recreate the dictionary
    p0_dict = {key: val for key, val in zip(p0_names, p0)}

    # HST Phase
    HSTphase = np.zeros(nexposure)
    HSTphase = (img_date - p0_dict['T0']) / HST_period
    phase2 = np.floor(HSTphase)
    HSTphase = HSTphase - phase2
    k = np.where(HSTphase > 0.5)[0]

    if k[0].shape == 0:
    #if k[0] != -1:         # in IDL this meant if condition of "where" statement is true nowhere
        HSTphase[k] = HSTphase[k] - 1.0

    phase = np.zeros(nexposure)
    for j in range(nexposure):
        phase[j] = (img_date[j] - p0_dict['epoch']) / (p0_dict['Per'] / day_to_sec)

    phase2 = np.floor(phase)
    phase = phase - phase2
    a = np.where(phase > 0.5)[0]
    if a[0].shape == 0:
    #if a[0] != -1:
        phase[a] = phase[a] - 1.0

    ###############
    # MPFIT - TWO #
    ###############

    parinfo = []

    for i, value in enumerate(p0):
        info = {'value': 0., 'fixed': 0, 'limited': [0, 0], 'limits': [0., 0.]}
        info['value'] = value
        info['fixed'] = systematics[i]
        parinfo.append(info)

    fa = {'x': img_date, 'y': img_flux, 'err': err, 'sh': sh}
    mpfit_result = mpfit(hstmarg.transit_circle, functkw=fa, parinfo=parinfo, quiet=1)
    nfree = sum([not p['fixed'] for p in parinfo])
    # The python mpfit does not populate the covariance matrix correctly so m.perror is not correct
    pcerror = mpfit_result.perror  # this is how it should be done if it was right
    pcerror = np.zeros_like(mpfit_result.perror)
    pcerror[:nfree] = np.sqrt(
        np.diag(mpfit_result.covar.flatten()[:nfree ** 2].reshape(nfree, nfree)))  # this might work...

    # From mpfit define the DOF, BIC, AIC & CHI
    bestnorm = mpfit_result.fnorm  # chi squared of resulting fit
    BIC = bestnorm + nfree * np.log(len(img_date))
    AIC = bestnorm + nfree
    DOF = len(img_date) - sum([p['fixed'] != 1 for p in parinfo])  # nfree
    CHI = bestnorm

    # EVIDENCE BASED on the AIC and BIC
    Mpoint = nfree
    Npoint = len(img_date)
    sigma_points = np.median(err)

    evidence_BIC = - Npoint * np.log(sigma_points) - 0.5 * Npoint * np.log(2 * np.pi) - 0.5 * BIC
    evidence_AIC = - Npoint * np.log(sigma_points) - 0.5 * Npoint * np.log(2 * np.pi) - 0.5 * AIC

    # Redefine all of the parameters given the MPFIT output
    # Redefine array
    res_sec = mpfit_result.params
    # Recreate the dictionary
    res_sec_dict = {key: val for key, val in zip(p0_names, res_sec)}

    # pcerror = [rl_err, flux0_err, epoch_err, inclin_err, msmpr_err, ecc_err, omega_err, per_err, T0_err,
    #           c1_err, c2_err, c3_err, c4_err, m_err, HSTP1_err, HSTP2_err, HSTP3_err, HSTP4_err, xshift1_err,
    #           xshift2_err, xshift3_err, xshift4_err]
    rl_err = pcerror[0]
    epoch_err = pcerror[2]

    # Recalculate a/R* (actually the constant for it) based on the new MsMpR value which may have been fit in the routine.
    constant1 = (Gr * res_sec_dict['Per'] * res_sec_dict['Per'] / (4 * np.pi * np.pi)) ** (1 / 3.)
    aval = constant1 * (res_sec_dict['MsMpR']) ** (1 / 3.)   # NOT-REUSED

    print('\nTRANSIT DEPTH rl in model {} of {} = {} +/- {}     centered at  {}'.format(s+1, nsys, res_sec_dict['rl'], rl_err, res_sec_dict['epoch']))

    # OUTPUTS
    # Re-Calculate each of the arrays dependent on the output parameters for the epoch
    phase = (img_date - res_sec_dict['epoch']) / (res_sec_dict['Per'] / day_to_sec)
    phase2 = np.floor(phase)
    phase = phase - phase2
    a = np.where(phase > 0.5)[0]
    if len(a) > 0:
        phase[a] = phase[a] - 1.0

    HSTphase = (img_date - res_sec_dict['T0']) / HST_period
    phase2 = np.floor(HSTphase)
    HSTphase = HSTphase - phase2
    k = np.where(HSTphase > 0.5)[0]
    if len(k) > 0:
        HSTphase[k] = HSTphase[k] - 1.0

    # ...........................................
    # TRANSIT MODEL fit to the data
    # Calculate the impact parameter based on the eccentricity function
    b0 = hstmarg.impact_param(res_sec_dict['Per'], res_sec_dict['MsMpR'], phase, res_sec_dict['inclin'])

    mulimb01, mulimbf1 = hstmarg.occultnl(res_sec_dict['rl'], res_sec_dict['c1'], res_sec_dict['c2'], res_sec_dict['c3'], res_sec_dict['c4'], b0)

    # ...........................................
    # SMOOTH TRANSIT MODEL across all phase
    # Calculate the impact parameter based on the eccentricity function
    x2 = np.arange(4000) * 0.0001 - 0.2
    b0 = hstmarg.impact_param(res_sec_dict['Per'], res_sec_dict['MsMpR'], x2, res_sec_dict['inclin'])

    mulimb02, mulimbf2 = hstmarg.occultnl(res_sec_dict['rl'], res_sec_dict['c1'], res_sec_dict['c2'], res_sec_dict['c3'], res_sec_dict['c4'], b0)

    systematic_model = hstmarg.sys_model(phase, HSTphase, sh, res_sec_dict['m_fac'], res_sec_dict['HSTP1'], res_sec_dict['HSTP2'],
                                         res_sec_dict['HSTP3'], res_sec_dict['HSTP4'], res_sec_dict['xshift1'], res_sec_dict['xshift2'],
                                         res_sec_dict['xshift3'], res_sec_dict['xshift4'])

    fit_model = mulimb01 * res_sec_dict['flux0'] * systematic_model
    residuals = (img_flux - fit_model) / res_sec_dict['flux0']
    resid_scatter = np.std(w_residuals)
    fit_data = img_flux / (res_sec_dict['flux0'] * systematic_model)
    fit_err = np.copy(err)  # * (1.0 + resid_scatter)

    if plotting:
        plt.figure(2)
        plt.clf()
        plt.scatter(phase, img_flux, s=5)
        plt.plot(x2, mulimb02, 'k')
        plt.errorbar(phase, fit_data, yerr=err, fmt='m.')
        plt.xlim(-0.03, 0.03)
        plt.title('Model ' + str(s+1) + '/' + str(nsys))
        plt.xlabel('Planet Phase')
        plt.ylabel('Data')
        plt.draw()
        plt.pause(0.05)

    # .............................
    # Fill info into arrays to save to file once we iterated through all systems with both fittings.

    sys_stats[s, :] = [AIC, BIC, DOF, CHI, resid_scatter]   # stats
    sys_date[s, :] = img_date                               # input time data (x, date)
    sys_phase[s, :] = phase                                 # phase
    sys_rawflux[s, :] = img_flux                                   # raw lightcurve flux
    sys_rawflux_err[s, :] = err
    sys_flux[s, :] = fit_data                               # corrected lightcurve flux
    sys_flux_err[s, :] = fit_err
    sys_residuals[s, :] = residuals                         # residuals
    sys_model[s, :] = mulimb02                              # smooth model
    sys_model_phase[s, :] = x2                              # smooth phase
    sys_systematic_model[s, :] = systematic_model           # systematic model
    sys_params[s, :] = mpfit_result.params                  # parameters
    sys_params_err[s, :] = pcerror                          # parameter errors
    sys_depth[s] = res_sec_dict['rl']                            # depth
    sys_depth_err[s] = rl_err                               # depth error
    sys_epoch[s] = res_sec_dict['epoch']                         # transit time
    sys_epoch_err[s] = epoch_err                            # transit time error
    sys_evidenceAIC[s] = evidence_AIC                       # evidence AIC
    sys_evidenceBIC[s] = evidence_BIC                       # evidence BIC

    print('Another round done')

# Save to file
# For details on how to deal with this kind of file, see the notebook "NumpyData.ipynb"
np.savez(os.path.join(outDir, 'analysis_circle_G141_'+run_name), sys_stats=sys_stats, sys_date=sys_date, sys_phase=sys_phase,
         sys_rawflux=sys_rawflux, sys_rawflux_err=sys_rawflux_err, sys_flux=sys_flux, sys_flux_err=sys_flux_err,
         sys_residuals=sys_residuals, sys_model=sys_model, sys_model_phase=sys_model_phase,
         sys_systematic_model=sys_systematic_model, sys_params=sys_params, sys_params_err=sys_params_err,
         sys_depth=sys_depth, sys_depth_err=sys_depth_err, sys_epoch=sys_epoch, sys_epoch_err=sys_epoch_err,
         sys_evidenceAIC=sys_evidenceAIC, sys_evidenceBIC=sys_evidenceBIC)


In [ ]:
test = mpfit_result.covar.flatten()[:nfree**2].reshape(nfree, nfree)
ma = pd.DataFrame(test)
print(ma)

In [ ]:
tri = np.sqrt(np.diag(test))

In [ ]:
print(type(tri))
print(tri)

In [ ]:
ind = np.where(systematics == 0)[0]
pcerror[ind] = tri

In [ ]:
pcerror[ind]

In [ ]:
pcerror